In [1]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## RunnablePassthrough

* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
chain.invoke("Abhi")

'Abhi'

## RunnableLambda

* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.

In [5]:
# Let's define a very simple function to create Russian lastnames
def russain_lastname(name: str) -> str:
    return f"{name}ovich"

In [6]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russain_lastname)

chain.invoke("Abhi")

'Abhiovich'

## RunnableParallel

* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.

In the following chain, RunnableParallel is going to run these two tasks in parallel:
* operation_a will use RunnablePassthrough.
* operation_b will use RunnableLambda with the russian_lastname function.

In [7]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russain_lastname)
    }
)

chain.invoke("Abhi")

{'operation_a': 'Abhi', 'operation_b': 'Abhiovich'}

In [8]:
# another example
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}.")

output_parser = StrOutputParser()

def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough()
    }
) | prompt | model | output_parser

chain.invoke(
    {
        "name1": "Jordam",
        "name": "Abram"
    }
)

"Roman Abramovich, the Russian billionaire and owner of Chelsea Football Club, once owned the world's largest private yacht. Named Eclipse, the yacht cost an estimated $1.5 billion to build and is equipped with two swimming pools, a helicopter pad, a submarine, and even a missile defense system."

### More advanced use of RunnableParallel

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


vectorstore = FAISS.from_texts(
    ["Abhi is pursuing Master's in Data Science at IU International University of Applied Sciences. He is staying in Berlin, Germany. He has 6 years of work experience as an IT Professional in India. He is also an AI enthusiast and has a keen interest in AI and ML. He is also a data science enthusiast and has a keen interest in data science."],
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based on the following context: 
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is Abhi's work experience?")

'Abhi has 6 years of work experience as an IT Professional in India.'

In [12]:
# another example
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


model = ChatOpenAI(model="gpt-3.5-turbo-0125")

vectorstore = FAISS.from_texts(
    ["Abhi is pursuing Master's in Data Science at IU International University of Applied Sciences. He is staying in Berlin, Germany. He has 6 years of work experience as an IT Professional in India. He is also an AI enthusiast and has a keen interest in AI and ML. He is also a data science enthusiast and has a keen interest in data science."],
    embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language")
    }
    | prompt | model | StrOutputParser()
)

chain.invoke({"question": "What is Abhi's work experience?", "language": "German"})

'Abhi hat 6 Jahre Berufserfahrung als IT-Professional in Indien.'